# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape, 'y_height is', image.shape[0], 'x_width is', image.shape[1])
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [7]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #Calculates the per-element bit-wise conjunction of two arrays or an array and a scalar.
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

# Note: To disable extrapolate to print segmented lines, please set it to False
def draw_lines(img, lines, color=[255, 0, 0], thickness=12, extrapolate=True):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    # Algorithm:
    # 1. Divide the line segments into two parts: L&R
    # 2. Calculate the mean slope and intercept
    # 3. We need four vertices(two points) to draw a line
    #    -(X1, Y2) & (X2, Y2) -->calculate that!
    # 4. Draw the two lines
    # Note: my signature function with input argument to invoke extrapolate
    
    # 0. Initialization 
    # y=mx+b, m is slope
    reference_m = 0.65 #
    buffer = 0.30
    line_ref_m_min = reference_m * (1 - buffer) # ~=0.46
    line_ref_m_max = reference_m * (1 + buffer) # ~=0.85
    left_lines, right_lines = [], []
    left_lines_m, right_lines_m = [], []
    left_lines_b, right_lines_b = [], []
    left_lines_x, right_lines_x = [], []
    avg_left_lines_m, avg_right_lines_m = 0, 0
    avg_left_lines_b, avg_right_lines_b = 0, 0
    max_left_lines_x, min_right_lines_x = 0, 0
    
    # 1. Divide the lines & Collect targeted data points for m & b
    for line in lines:
        for x1,y1,x2,y2 in line:
            # Define m and check for Division by Zero
            try:
                m = (y2-y1)/(x2-x1)
            except ZeroDivisionError:
                m = -np.inf
            
            # Detect left line whose m < 0
            # Scope down the targeted lines using reference slope
            if m < 0: # This step is key to filter out the invalid lines with positive m!
                if abs(m) >= line_ref_m_min and abs(m) <= line_ref_m_max:
                    left_lines.append(line)
                    left_lines_m.append(m)
                    left_lines_b.append(y1-m*x1)
                    left_lines_x.append(x1)
                    left_lines_x.append(x2)
                
            # Detect right line whose m > 0
            elif m >= line_ref_m_min and m <= line_ref_m_max:
                right_lines.append(line)
                right_lines_m.append(m)
                right_lines_b.append(y1-m*x1)
                right_lines_x.append(x1)
                right_lines_x.append(x2)
    
    if(extrapolate == False):
        # Just draw the segmented lines
        for line in left_lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)

        for line in right_lines:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
    if(extrapolate == True):   
        # 2. Calculate the mean slope and intercept        
        if (len(left_lines_m) > 0):
            avg_left_lines_m = np.mean(left_lines_m)
            avg_left_lines_b = np.mean(left_lines_b)
            max_left_lines_x = np.amax(left_lines_x)

        # Debug due to division by zero error
        # from IPython.core.debugger import Tracer; Tracer()()
        if (len(right_lines_m) > 0):
            avg_right_lines_m = np.mean(right_lines_m)
            avg_right_lines_b = np.mean(right_lines_b)
            min_right_lines_x = np.amin(right_lines_x)
        
        # For overcoming Division by Zero error 
        if (avg_left_lines_m == 0):
            avg_left_lines_m += 0.000001
        if (avg_right_lines_m == 0):
            avg_right_lines_m += 0.000001
        
        # 3. Find two points
        # To feed the function cv2.line, we need two points, since we got y=m*x+b,
        # we can use some special points to find the two points (Xmin,Ymin) & (Xmax,Ymax).
        # Observe that the two lines intersect with the Ymax-the img height
        # int() for all the points due to cv2.line requires int as input
        # Weakness of this code block is Division by Zero
        left_line_x1 = int((img.shape[0]-avg_left_lines_b)/(avg_left_lines_m))
        left_line_y1 = int(img.shape[0])
        left_line_x2 = int(max_left_lines_x)
        left_line_y2 = int(avg_left_lines_m*max_left_lines_x+avg_left_lines_b)

        right_line_x1 = int((img.shape[0]-avg_right_lines_b)/(avg_right_lines_m))
        right_line_y1 = int(img.shape[0])
        right_line_x2 = int(min_right_lines_x)
        right_line_y2 = int(avg_right_lines_m*min_right_lines_x+avg_right_lines_b)
        
        # 4. Draw lines cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        # line_image = np.copy(image)*0  # creating a blank to draw lines on
        cv2.line(img, (left_line_x1, left_line_y1), (left_line_x2, left_line_y2), color, thickness)
        cv2.line(img, (right_line_x1, right_line_y1), (right_line_x2, right_line_y2), color, thickness)


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    Combine line image with original image to see how accurate the line annotations are.
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [3]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
import numpy as np 
import cv2
import os

def detect_lane_lines(img):
    
    # Algorithm:
    # 1. Transform RGB to grayscale using cv2.cvtColor()
    # 2. Apply Gaussian smoothing using cv2.GaussianBlur()
    # 3. Perform Canny Edge Detection using cv2.Canny()
    # 4. Create a masked edges image with defined region of interest 
    #    using cv2.fillPoly() & cv2.bitwise_and()
    # 5. Retrieve Hough lines using cv2.HoughLinesP() 
    #    which will apply lines to the original image using cv2.line()
    # 6. Draw the lines on the edge image using cv2.addWeighted()


    # 1. Read in and grayscale the image 
    # image = mpimg.imread('test_images/solidWhiteRight.jpg')
    gray = grayscale(img) 

    # 2. Define a kernel size and apply Gaussian smoothing 
    kernel_size = 5 #3
    blur_gray = gaussian_blur(gray, kernel_size) 

    # 3. Define our parameters for Canny and apply 
    low_threshold = 50 # 55,75
    high_threshold = 150 # 165,180
    edges = canny(blur_gray, low_threshold, high_threshold) 

    # 4. Define region of interest - a four sided polygon to mask
    imgshape = img.shape
    alpha = 0.20 # Use to adjust the orientation of this trapezoid 
    vertices = np.array([[(imgshape[1]*alpha/2,imgshape[0]),
                          (imgshape[1]/2*(1-alpha), imgshape[0]/2*(1+alpha)), 
                          (imgshape[1]/2*(1+alpha), imgshape[0]/2*(1+alpha)), 
                          (imgshape[1]*(1-alpha/2),imgshape[0])]], 
                          dtype=np.int32) 
    region = region_of_interest(edges, vertices) 

    # 5.1 Define the Hough transform parameters 
    rho = 1 #2
    theta = np.pi/180 
    threshold = 15  # 40,100minimum number of votes (intersections in Hough grid cell) 
    min_line_length = 40 # 20,40,75,25minimum number of pixels making up a line 
    max_line_gap = 20 # 10,20,50,25maximum gap in pixels between connectable line segments 
 
    # Optional. Create a blank to draw lines on
    # line_image = np.copy(image)*0 

    # 5.2 Run Hough Transform on edge detected image 
    # Output "lines" is an array containing endpoints of detected line segments 
    lines = hough_lines(region, 
                        rho, 
                        theta, 
                        threshold,  
                        min_line_length, 
                        max_line_gap) 

    # Optional. Create a "color" binary image to combine with line image 
    # color_edges = np.dstack((edges, edges, edges)) 
    #ignore_color = np.copy(masked_lines)*0 # creating a blank color channel for combining
    #line_image = np.dstack((masked_lines, ignore_color, ignore_color)) 

    # 6. Draw the lines on the edge image 
    # lines_edges = weighted_img(lines, line_image) 
    lines_edges = weighted_img(lines, img)
    return lines_edges


In [ ]:

# Instructions to run:
# 1. For detecting segmented lines, just switch the toggle "extrapolate" of function detect_lane_lines to False, 
#    change the "Extrapolated" and "extrapolated" words to "Processed" and "processed", respectively, 
#    then execute the following script.
#
# 2. For extrapolating the segmented lines, switch the toggle "extrapolate" of function detect_lane_lines to True,
#    change the "Processed" and "processed" words to "Extrapolated" and "extrapolated", respectively,
#    then execute the following script.
#
# Create test_images_output directory per instruction
if not os.path.exists("test_images_output"):
    os.makedirs("test_images_output")

for test_image in os.listdir("test_images/"):
    image = mpimg.imread("test_images/" + test_image)
    lines_edges = detect_lane_lines(image)
    # Save them to the test_images_output directory.
    # mpimg.imsave("test_images_output/" + "Extrapolated_" + test_image , lines_edges)
    # Display the images
    plt.imshow(lines_edges)
    plt.title(test_image)
    plt.show()
    print("Image: " + test_image + " is extrapolated.")
print("Done!")


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [5]:
def process_image(img):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # return detect_lane_lines(img) --> This does not work, so follow the TODO to copy!!!
    # Copy all the code from function detect_lane_lines() here to fix the frame stuck problem
    
    #-----------------Copy/Paste from detect_lane_lines()------------
    # Algorithm:
    # 1. Transform RGB to grayscale using cv2.cvtColor()
    # 2. Apply Gaussian smoothing using cv2.GaussianBlur()
    # 3. Perform Canny Edge Detection using cv2.Canny()
    # 4. Create a masked edges image with defined region of interest 
    #    using cv2.fillPoly() & cv2.bitwise_and()
    # 5. Retrieve Hough lines using cv2.HoughLinesP() 
    #    which will apply lines to the original image using cv2.line()
    # 6. Draw the lines on the edge image using cv2.addWeighted()

    # Try to resize the resolution to avoid Division by Zero error!-->not working!
    # img = cv2.resize(img,(960,540))
    
    # 1. Read in and grayscale the image 
    # image = mpimg.imread('test_images/solidWhiteRight.jpg')
    gray = grayscale(img) 

    # 2. Define a kernel size and apply Gaussian smoothing 
    kernel_size = 5 #3
    blur_gray = gaussian_blur(gray, kernel_size) 

    # 3. Define our parameters for Canny and apply 
    low_threshold = 50 # 55,75
    high_threshold = 150 # 165,180
    edges = canny(blur_gray, low_threshold, high_threshold) 

    # 4. Define region of interest - a four sided polygon to mask
    imgshape = img.shape
    alpha = 0.20 # Use to adjust the orientation of this trapezoid 
    vertices = np.array([[(imgshape[1]*alpha/2,imgshape[0]),
                          (imgshape[1]/2*(1-alpha), imgshape[0]/2*(1+alpha)), 
                          (imgshape[1]/2*(1+alpha), imgshape[0]/2*(1+alpha)), 
                          (imgshape[1]*(1-alpha/2),imgshape[0])]], 
                          dtype=np.int32) 
    region = region_of_interest(edges, vertices) 

    # 5.1 Define the Hough transform parameters 
    rho = 1 #2
    theta = np.pi/180 
    threshold = 15  # 40,100minimum number of votes (intersections in Hough grid cell) 
    min_line_length = 40 # 20,40,75,25minimum number of pixels making up a line 
    max_line_gap = 20 # 10,20,50,25maximum gap in pixels between connectable line segments 
 
    # Optional. Create a blank to draw lines on
    # line_image = np.copy(image)*0 

    # 5.2 Run Hough Transform on edge detected image 
    # Output "lines" is an array containing endpoints of detected line segments 
    lines = hough_lines(region, 
                        rho, 
                        theta, 
                        threshold,  
                        min_line_length, 
                        max_line_gap) 

    # Optional. Create a "color" binary image to combine with line image 
    # color_edges = np.dstack((edges, edges, edges)) 
    # ignore_color = np.copy(masked_lines)*0 # creating a blank color channel for combining
    # line_image = np.dstack((masked_lines, ignore_color, ignore_color)) 

    # 6. Draw the lines on the edge image 
    # lines_edges = weighted_img(lines, line_image) 
    lines_edges = weighted_img(lines, img)
    return lines_edges

In [ ]:
# Create test_videos_output directory, which is missing from the instruction!
if not os.path.exists("test_videos_output"):
    os.makedirs("test_videos_output")

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [8]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:12<00:00, 19.51it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

Wall time: 13.7 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))